# Spark Recommender System

In [4]:
import os
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

spark = SparkSession.builder.appName('rec').getOrCreate()

PATH = os.path.join(os.getcwd(),"Python-and-Spark-for-Big-Data-master/Spark_for_Machine_Learning/Recommender_Systems/")

data = spark.read.csv(os.path.join(PATH,'movielens_ratings.csv'),inferSchema=True, header=True)

In [5]:
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [6]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [7]:
train, test = data.randomSplit([0.8,0.2])

In [8]:
als = ALS(maxIter=5,regParam=0.01,userCol='userId',itemCol='movieId',ratingCol='rating')

In [9]:
model = als.fit(train)

In [10]:
predictions = model.transform(test)

In [11]:
predictions.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|     31|   1.0|    26|  0.7486136|
|     31|   1.0|    27|-0.07873018|
|     31|   4.0|    12|  2.4258802|
|     31|   1.0|    13|  0.6698139|
|     31|   3.0|    14|  2.7957733|
|     85|   1.0|    28|  1.8002945|
|     85|   5.0|    16|  2.9649851|
|     85|   2.0|    20|0.019636916|
|     85|   1.0|    15|  2.5702257|
|     65|   1.0|    28|  1.8312372|
|     65|   1.0|    22|  1.4416679|
|     65|   2.0|     3|  1.6014229|
|     65|   1.0|    19|  1.2562511|
|     65|   1.0|    24|  1.5967927|
|     34|   1.0|    19|  1.5571206|
|     34|   1.0|    15| 0.25887716|
|     81|   5.0|    28|  1.0695119|
|     81|   3.0|    26|  1.0733606|
|     81|   1.0|    16|  1.5759932|
|     81|   2.0|    29| 0.17854084|
+-------+------+------+-----------+
only showing top 20 rows



In [12]:
evaluator = RegressionEvaluator(metricName='rmse',labelCol='rating',predictionCol='prediction')

In [13]:
rmse = evaluator.evaluate(predictions)

In [14]:
print(f"RMSE: {rmse}")

RMSE: 1.6653016832731247


In [15]:
single_user = test.filter(test['userId']==11).select(['movieId','userId'])

In [16]:
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      9|    11|
|     18|    11|
|     35|    11|
|     38|    11|
|     40|    11|
|     50|    11|
|     77|    11|
+-------+------+



In [17]:
recommendations = model.transform(single_user)

In [18]:
recommendations.orderBy('prediction',ascending=False).show()

+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
|     18|    11| 3.6077456|
|     50|    11| 2.9048553|
|     40|    11| 2.7517812|
|      9|    11| 1.3795893|
|     38|    11| 0.9291505|
|     35|    11|0.37590984|
|     77|    11| 0.2269172|
+-------+------+----------+

